In [1]:
import train
import logging
import torch
from EfficientUnet import EfficientUnet
import importlib
from datasets import BasicDataset
import matplotlib.pyplot as plt
import random

In [2]:
from matplotlib.colors import ListedColormap

cmap = ListedColormap([[0.29411764705882354, 0.4392156862745098, 0.7333333333333333],
                        [0.5882352941176471, 0.7607843137254902, 0.8666666666666667],
                        [0.8901960784313725, 0.9647058823529412, 0.9764705882352941],
                        [0.9803921568627451, 0.8745098039215686, 0.4666666666666667],
                        [0.9607843137254902, 0.47058823529411764, 0.29411764705882354],
                        [0.8470588235294118, 0.1568627450980392, 0.1411764705882353]]
                            )

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
model = EfficientUnet(n_classes=6).to(device)

logging.info(f'Network:\n'
        f'\t{model.n_channels} input channels\n'
        f'\t{model.n_classes} output channels (classes)\n')


INFO: Using device cuda
INFO: Network:
	2 input channels
	6 output channels (classes)



In [4]:
train_dir = 'data/images/train'
train_mask_dir = 'data/annotations/train'
val_dir = 'data/images/val'
val_mask_dir = 'data/annotations/val'
test_dir = 'data/images/test'
test_mask_dir = 'data/annotations/test'
checkpoint_dir = 'checkpoints/'

In [5]:
test_dataset = BasicDataset(test_dir, test_mask_dir, augmentation=False)
# tese_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, pin_memory=True)

INFO: Creating dataset with 400 examples
INFO: Scanning mask files to determine unique values
100%|██████████| 400/400 [00:00<00:00, 3642.41it/s]
INFO: Unique mask values: [0, 1, 2, 3, 4, 5]


In [7]:
importlib.reload(train)

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 10
model = EfficientUnet(n_classes=6).to(device)
torch.cuda.empty_cache()

train.train_model(
    model=model,
    device=device,
    dir_img_train=train_dir,
    dir_mask_train=train_mask_dir,
    dir_img_val=val_dir,
    dir_mask_val=val_mask_dir,
    dir_checkpoint=checkpoint_dir,
    save_checkpoint=True,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    val_percent=0.1,
    augmentation=True,
    gradient_clipping=1.0,
)


[701, 401]


TypeError: list.append() takes exactly one argument (701 given)

In [ ]:
idx = random.randint(0, len(test_dataset))

instancia = test_dataset[idx]

data_teste, mask_teste = instancia['image'], instancia['mask']

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

model.eval()
with torch.no_grad():
    data_teste = data_teste.unsqueeze(0).to(device).float()
    mask_pred = model(data_teste)
    mask_pred = mask_pred.squeeze(0).cpu()
    mask_pred = torch.argmax(mask_pred, dim=0)

axs[0].imshow(mask_teste, cmap=cmap)
axs[1].imshow(mask_pred, cmap=cmap)

# Remove the axis labels
for ax in axs:
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
from torchmetrics.classification import MulticlassJaccardIndex, MulticlassF1Score

IoU = MulticlassJaccardIndex(num_classes=model.n_classes).to(device=device)
F1_score = MulticlassF1Score(num_classes=model.n_classes).to(device=device)

In [ ]:
from utils import get_input_data, load_image
import numpy as np
import torch

cp_num = EPOCHS
# model.load_state_dict(torch.load(f'checkpoints/checkpoint{cp_num}.pth', map_location=device))
# model_teste = EfficientUnet(n_classes=6).to(device)
# model_teste.load_state_dict(torch.load(f'checkpoints/checkpoint{cp_num}.pth', map_location=device))

il_list = [32, 109]
masks = []
preds = []
lista = []
fig, axs = plt.subplots(len(il_list), 2, figsize=(12, 4))

for i, il in enumerate(il_list):
    data_path = 'data/images/test/il_{}.tif'.format(il)
    mask_path = 'data/annotations/test/il_{}.png'.format(il)
    
    data_teste = get_input_data(data_path)
    mask_teste = load_image(mask_path)
    data_teste = torch.tensor(data_teste).unsqueeze(0).to(device).permute(0, 3, 1, 2).float()
    
    mask_pred = model(data_teste)
    mask_pred = mask_pred.squeeze(0).cpu()
    mask_pred = torch.argmax(mask_pred, dim=0)[:mask_teste.shape[0], :mask_teste.shape[1]]
    print(data_teste.shape, mask_teste.shape, mask_pred.shape)
    
    difference = np.where(mask_teste == np.array(mask_pred), 1, 0)    
    
    axs[0, i].imshow(mask_teste, cmap=cmap)
    axs[1, i].imshow(mask_pred, cmap=cmap)
    # axs[i, 1].set_title('Inline {}'.format(il))
    # axs[i, 2].imshow(difference, cmap='gray')
    plt.tight_layout()

    for ax in axs[i]:
        ax.axis('off')
    
    lista.append(difference)
    masks.append(mask_teste)
    preds.append(mask_pred)


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].imshow(lista[0], cmap='grey')
axs[1].imshow(lista[1], cmap='grey')
axs[0].axis('off')
axs[1].axis('off')

In [ ]:
for i, img in enumerate(preds):
    iou = IoU(img.unsqueeze(0).to(device), torch.tensor(masks[i]).unsqueeze(0).to(device))
    f1 = F1_score(img.unsqueeze(0).to(device), torch.tensor(masks[i]).unsqueeze(0).to(device))
    print(f'Inline {il_list[i]} -> IoU: {iou:.2f}; F1: {f1:.2f}')

In [ ]:
test_dataset = BasicDataset(test_dir, test_mask_dir, augmentation=False)
teste_dl = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

In [ ]:
from evaluate import evaluate_model

iou, f1, loss = evaluate_model(model, teste_dl, device, window=False)
print(f'IoU: {iou:.2f}; F1: {f1:.2f}; Loss: {loss:.2f}')